# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [1]:
import torch as th
import syft as sy

In [2]:
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id = 'bob')
alice = sy.VirtualWorker(hook, id = 'alice')

In [3]:
bob.clear_objects()

<VirtualWorker id:bob #tensors:0>

In [4]:
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [5]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [6]:
x

(Wrapper)>[PointerTensor | me:45337007723 -> bob:53762407734]

In [7]:
bob._objects

{53762407734: tensor([1, 2, 3, 4, 5])}

In [8]:
x = x.send(alice)

In [9]:
bob._objects

{53762407734: tensor([1, 2, 3, 4, 5])}

In [10]:
alice._objects

{45337007723: (Wrapper)>[PointerTensor | alice:45337007723 -> bob:53762407734]}

In [11]:
x

(Wrapper)>[PointerTensor | me:84992736991 -> alice:45337007723]

In [12]:
y = x + x

In [13]:
y

(Wrapper)>[PointerTensor | me:67978351427 -> alice:77583698621]

In [14]:
bob._objects

{53762407734: tensor([1, 2, 3, 4, 5]),
 99834321029: tensor([ 2,  4,  6,  8, 10])}

In [15]:
alice._objects

{45337007723: (Wrapper)>[PointerTensor | alice:45337007723 -> bob:53762407734],
 77583698621: (Wrapper)>[PointerTensor | alice:77583698621 -> bob:99834321029]}

In [16]:
jon = sy.VirtualWorker(hook, id="jon")

In [19]:
x = th.tensor([1,2,3,4,5]).send(bob).send(jon)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [22]:
z = x + y
z.get()

(Wrapper)>[PointerTensor | me:18318927781 -> bob:45535563194]

In [26]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [27]:
bob._objects

{254959992: tensor([1, 2, 3, 4, 5])}

In [28]:
alice._objects

{9442114436: (Wrapper)>[PointerTensor | alice:9442114436 -> bob:254959992]}

In [29]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:9442114436 -> bob:254959992]

In [30]:
bob._objects

{254959992: tensor([1, 2, 3, 4, 5])}

In [31]:
alice._objects

{}

In [32]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

In [33]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [34]:
bob._objects

{21463460523: tensor([1, 2, 3, 4, 5])}

In [35]:
alice._objects

{3923429312: (Wrapper)>[PointerTensor | alice:3923429312 -> bob:21463460523]}

In [36]:
del x

In [37]:
bob._objects

{}

In [38]:
alice._objects

{}